In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score
from sklearn.metrics import balanced_accuracy_score, precision_recall_curve
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn import preprocessing
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.metrics import classification_report
import seaborn as sns 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import  mutual_info_classif 
from sklearn.model_selection import GridSearchCV
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [85]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [86]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt
#!pip install scikit-learn  -U
SEED = 100

### eda


### Названия признаков и их смысл:
    
    client_id - идентификатор клиента
    education - уровень образования
    sex - пол заемщика
    age - возраст заемщика
    car - флаг наличия автомобиля
    car_type - флаг автомобиля иномарки
    decline_app_cnt - количество отказанных прошлых заявок
    good_work - флаг наличия “хорошей” работы
    bki_request_cnt - количество запросов в БКИ
    home_address - категоризатор домашнего адреса
    work_address - категоризатор рабочего адреса
    income - доход заемщика
    foreign_passport - наличие загранпаспорта
    sna - связь заемщика с клиентами банка
    first_time - давность наличия информации о заемщике
    score_bki - скоринговый балл по данным из БКИ
    region_rating - рейтинг региона
    app_date - дата подачи заявки
    default - флаг дефолта по кредиту

In [87]:
# Размеры выборок
df_train.shape, df_test.shape, sample_submission.shape

In [88]:
#Обьединяем данные для оценки признаков

In [89]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения default, мы его должны предсказать, поэтому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [90]:
data.info()

In [91]:
#Взглянем на данные
data.head(10)

In [92]:
#Оценим количество пропущенных значений
data.isnull().sum()

In [93]:
# Поскольку пропусков незначительное количество , сразу же их обработем , заменив их на новое значение и превратив в бинарные переменные 
data['education'] =data['education'].fillna('Оmission')
data2=pd.get_dummies(data.education, drop_first=False)
data=pd.concat([data,data2],sort=False,axis=1)
data.drop(columns = ['education'],axis = 1, inplace=True)
data.head(10)

In [94]:
# Посморим количество уникальных значений
data.nunique(dropna=False)

In [95]:
# так же выделим столбцы по категориям 
#бинарные переменные 
bin_cols = ['ACD','GRD','PGR','SCH','UGR','Оmission','sex', 'car', 'car_type', 'good_work', 'foreign_passport']

#категориальные переменные
cat_cols =  [ 'work_address', 'home_address', 'region_rating', 'sna', 'first_time']

#числовые переменные

num_cols = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income']


In [96]:
data2= data[['work_address', 'home_address', 'region_rating', 'sna', 'first_time','age', 'decline_app_cnt', 'bki_request_cnt', 'income','sex', 'car', 'car_type', 'good_work', 'foreign_passport','default']]
sns.pairplot(data2, hue="default")

In [97]:
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
cat_cols = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['ACD','GRD','PGR','SCH','UGR','Оmission','sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [98]:
# Отберем признаки которые видимо влияют на распределения дефолта и посмотрим поближе 
# work_address
# home_address
# region_rating
# sna
# age
# good_work
# income
# Так же мы добавим ось доходов так как есть зависимость шанса банкротсва от доходов

In [99]:
# Здесь мы видим что чем region rating  меньше и income(зарплата) тем больше представлен default 1 (дефолт)  
sns.relplot(data=data, y = 'income', x = 'region_rating', kind='scatter', col='default')

In [100]:
# На этих графиках видно что с увеличением  возраста вероятность дефолта падает
sns.relplot(data=data, y = 'income', x = 'age', kind='scatter', col='default')

In [101]:
# Тут наблюдается что при определенных значенияx sna(2.0 и 3.0) и определенных доходах(около 0.4) вероятность дефолта увеличееваеться
sns.relplot(data=data, y = 'income', x = 'sna', kind='scatter', col='default')

In [102]:
# У людей с плохой работой и хорошими доходами дефолтов больше чем у людей с небольшими доходами но хорошей работой
sns.relplot(data=data, y = 'income', x = 'good_work', kind='scatter', col='default')

In [103]:
# Work address 3 и любые доходы повышают шанс дефолта
sns.relplot(data=data, y = 'income', x = 'work_address', kind='scatter', col='default')

In [104]:
# Home address 3 и маленькие доходы повышают вероятность дефолта
sns.relplot(data=data, y = 'income', x = 'home_address', kind='scatter', col='default')

In [105]:
# Взглянем на корреляцию числовых переменных
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1)

In [106]:
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [107]:
# Посмотрим, как обозначены данные в бинарных признаках и сколько их

for column in data[bin_cols]:
    print(data[column].value_counts(),  '\n'*2)

In [108]:
# Перекодируем бинарные признаки из буквенных обозначений в цифровые
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    print(column, dict(enumerate(label_encoder.classes_)))
    

data.sample()

In [109]:
# Посмотрим, сколько каких данных в других категориальных признаках
# Распределение неравномерное

data[['first_time', 'sna', 'work_address', 'home_address', 'region_rating']].hist(figsize=(8, 8))

In [110]:
#Значимость категориальных переменных
imp_cat = pd.Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'], discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

### Обработка признаков

In [111]:
data.drop(['client_id','app_date'], axis = 1, inplace=True)


In [112]:
# логарифмируем признак age
data['age'] = np.log(data['age'] + 1)

# логарифмируем признак decline_app_cnt
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)

# логарифмируем признак income
data['income'] = np.log(data['income'] + 1)
#data['income'] = data['income'].median()
# логарифмируем признак bki_request_cnt
data['bki_request_cnt'] = np.log(data.bki_request_cnt + 1)


In [113]:
data

In [114]:
data.info()
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
cat_cols = ['first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['ACD','GRD','PGR','SCH','UGR','Оmission','sex', 'car', 'car_type', 'good_work', 'foreign_passport']
bin_cols = ['ACD','GRD','PGR','SCH','UGR','Оmission','sex', 'car', 'car_type', 'good_work', 'foreign_passport']

## Model

In [115]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)
train, validation = train_test_split(train_data, test_size=0.3, random_state=SEED)

X_cat = OneHotEncoder(sparse = False).fit_transform(train[cat_cols].values)
X_num = StandardScaler().fit_transform(train[num_cols].values)
X_train = np.hstack([X_num, train[bin_cols].values, X_cat])
y_train = train['default'].values
X_cat = OneHotEncoder(sparse = False).fit_transform(validation[cat_cols].values)
X_num = StandardScaler().fit_transform(validation[num_cols].values)
X_validation = np.hstack([X_num, validation[bin_cols].values, X_cat])
y_true = validation['default'].values

In [116]:
validation.head(10)

In [117]:
#model = LogisticRegression(solver='liblinear', class_weight="balanced", max_iter=10000, random_state=SEED)
model = LogisticRegression(C =1.0, class_weight='balanced' ,dual= False,intercept_scaling=1,l1_ratio=None,max_iter=50,multi_class='auto',n_jobs=None,penalty="none",solver='sag',tol=0.02,verbose=0, random_state=SEED)


In [118]:
model.fit(X_train, y_train)

In [119]:
y_pred = model.predict(X_validation)

In [120]:
print(classification_report(y_true, y_pred ))


In [121]:
cm = confusion_matrix(y_true,y_pred)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [122]:
# Отрисовать ROC кривую
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from matplotlib import pyplot as plt

# Посчитать значения ROC кривой и значение площади под кривой AUC
fpr, tpr, thresholds = roc_curve(y_train,model.predict_proba(X_train).T[1])
roc_auc = roc_auc_score(y_train,model.predict_proba(X_train).T[1])   
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.title('Receiver Operating Characteristic', fontsize=15)
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.legend(fontsize=15)

In [123]:
X_cat = OneHotEncoder(sparse = False).fit_transform(test_data[cat_cols].values)
X_num = StandardScaler().fit_transform(test_data[num_cols].values)
X_train = np.hstack([X_num, test_data[bin_cols].values, X_cat])
predict_submission = model.predict(X_train)

In [124]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [125]:
sample_submission.describe()

In [126]:
sample_submission.to_csv('submission.csv', index=False)
#!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'